# Objectives

- To verify imputed cost for Rx encounters


In [92]:
## Import required libraries 

import snowflake.connector
import getpass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 400)
import plotly.graph_objects as go
pd.set_option("display.max_columns", 100)

In [93]:
import sys
sys.path.append('/home/jovyan/credentials')
import jupytertoolz_credentials as jt

In [94]:
## Snowflake tables 
ROLE = 'ANALYST'
WAREHOUSE = 'XLARGE_WH'
DATABASE = 'SANDBOX_KOMODO'  
SCHEMA = 'PROJECT_CURRENCY'

## Table reference
rx_version = '20220511'
rx_enc = f"MAP_ENCOUNTERS.RX_ENCOUNTERS_{rx_version}.RX_ENCOUNTER_LS_GA"
dm_version = '20220513'
dm_file = f"MAP_VOCABULARY.RXNORM_{dm_version}.DRUG_MASTER_ACTIVE_AND_HISTORICAL"

asp_price = 'ASP_PRICE_2022JAN'
asp_xwalk = 'ASP_NDC_HCPCS_CROSSWALK_2022JAN'
asp_noc_price = 'ASP_NOC_PRICE_2022JAN'
asp_noc_xwalk = 'ASP_NOC_NDC_HCPCS_CROSSWALK_2022JAN'
nadac_xwalk = 'NADAC_20220322'

sample = 1
seed = 1234

In [95]:
connection = snowflake.connector.connect(user='YWEI', password=getpass.getpass(), account='komodohealth')
connection.cursor().execute('USE ROLE ' + ROLE)
connection.cursor().execute('USE WAREHOUSE ' + WAREHOUSE)

def read_sql(sql, connection = connection):
    df = pd.read_sql(sql, connection)
    return df

def execute_sql(sql, connection = connection):
    #pd.io.sql.execute(sql, connection)
    connection.cursor().execute(sql)
    
execute_sql(f"USE ROLE ANALYST")
execute_sql(f"USE database {DATABASE}")
execute_sql(f"USE SCHEMA {SCHEMA}")

········


### Check List

1. NDC Validation 
2. distribution of days_supply, quantity, unit_price, cost
3. distribution of cost by year
4. frequency of imputation methods used
5. imputed vs actual
6. identify ndcs with cost outliers for further investigations 
7. route?
8. unit price comparison?

In [96]:
# define output table names - easier to rerun intermediate steps

# get all claims, filter to PAID and cleaning
rx_enc_step1 = f"RX_ENCOUNTERS_{rx_version}_STEP1"

# quantity imputation
days_supply_quantity_dist = f"RX_ENCOUNTERS_{rx_version}_DAYS_SUPPLY_QUANTITY_DIST"
days_supply_quantity_guess = f"RX_ENCOUNTERS_{rx_version}_DAYS_SUPPLY_QUANTITY_GUESS"
quantity_guess = f"RX_ENCOUNTERS_{rx_version}_QUANTITY_GUESS"
rx_enc_step2 = f"RX_ENCOUNTERS_{rx_version}_STEP2"

# unit price imputation
asp_per_unit_price = f"ASP_PER_UNIT_PRICE"
nadac_per_unit_price = f"NADAC_PER_UNIT_PRICE"
median_unit_price = f"RX_ENCOUNTERS_{rx_version}_MEDIAN_UNIT_PRICE"
unit_price = f"RX_ENCOUNTERS_{rx_version}_UNIT_PRICE"

# final imputation
cpi_index = "CPI_INDEX_MEDICALSERVICES_20220623"
cost_per_days_supply = f"RX_ENCOUNTERS_{rx_version}_COST_PER_DAYS_SUPPLY"
rx_enc_impute = f"RX_ENCOUNTERS_{rx_version}_IMPUTED"

In [ ]:
%%time
# create 1% sample for key datasets
# check NDC against DM, NADAC, ASP
sql = f"""
    create or replace table {rx_enc_step1}_1p as select * from {rx_enc_step1} sample (1) seed (1234);
"""
#execute_sql(sql)

sql = f"""
    create or replace table {rx_enc_step2}_1p as select * from {rx_enc_step2}
    where encounter_key in (select encounter_key from {rx_enc_step1}_1p);
"""
#execute_sql(sql)

sql = f"""
    create or replace table {rx_enc_impute}_1p as select * from {rx_enc_impute}
    where encounter_key in (select encounter_key from {rx_enc_step1}_1p);
"""
execute_sql(sql)

In [ ]:
# optional: run analysis using 1p tables
rx_enc_step1_1p = rx_enc_step1 + "_1p"
rx_enc_step2_1p = rx_enc_step2 + "_1p"
rx_enc_impute_1p = rx_enc_impute + "_1p"

In [ ]:
%%time
# NDC validation against DM, NADAC, ASP
sql = f"""
    with tmp as (
    select ndc,
        ndc in (select distinct ndc from {dm_file}) as in_dm,
        ndc in (select distinct ndc2 from {asp_xwalk}) as in_asp,
        ndc in (select distinct NDC_OR_ALTERNATE_ID from {asp_noc_xwalk}) as in_asp_noc,
        ndc in (select distinct ndc from {nadac_xwalk}) as in_nadac
    from {rx_enc_step1}_1p
    )
    select 
        count(*) as claim, 
        count_if(ndc is null or length(ndc) < 11 or left(ndc, 5) = '00000' or left(ndc, 5) = '99999') as claim_null, 
        count(distinct ndc) as code, 
        count_if(in_dm) as claim_dm,
        count(distinct (case when in_dm then ndc end)) as code_dm,
        count_if(in_asp) as claim_asp,
        count(distinct (case when in_asp then ndc end)) as code_asp,
        count_if(in_asp_noc) as claim_asp_noc,
        count(distinct (case when in_asp_noc then ndc end)) as code_asp_noc,
        count_if(in_nadac) as claim_nadac,
        count(distinct (case when in_nadac then ndc end)) as code_nadac,
        count_if(in_dm or in_nadac or in_asp or in_asp_noc) as claim_any,
        count(distinct (case when in_dm or in_nadac or in_asp or in_asp_noc then ndc end)) as code_any
    from tmp
    ;
"""
df_ndc = read_sql(sql)
display(df_ndc)

In [ ]:
def get_dist_cts(var, table, filter = 'TRUE', filter_print = None, group_by_list = None):
    if filter_print is None:
        filter_print = filter
    group_by_str = ''
    order_by_str = ''
    group_by_select_str = ''
    if group_by_list is not None:
        group_by = ",".join(group_by_list)
        group_by_select_str = group_by + ','
        group_by_str = 'group by ' + group_by
        order_by_str = 'order by ' + group_by
    return f"""
        select 
            {group_by_select_str}
            '{var}' as var, 
            '{filter_print}' as filter,
            count(*) as n,
            count_if({var} is null) as n_null,
            count_if({var} < 0) as n_neg,
            count_if({var} = 0) as n_zero,
            count_if({var} > 0) as n_pos,
            avg({var}) as mean,
            variance({var}) as variance,
            min({var}) as min,
            PERCENTILE_CONT(0.01) WITHIN GROUP (ORDER BY {var}) as p_01,
            PERCENTILE_CONT(0.05) WITHIN GROUP (ORDER BY {var}) as p_05,
            PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY {var}) as p_10,
            PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY {var}) as p_25,
            PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY {var}) as median,
            PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY {var}) as p_75,
            PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY {var}) as p_90,
            PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY {var}) as p_95,
            PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY {var}) as p_99,
            max({var}) as max
        from {table}
        where {filter}
        {group_by_str}
        {order_by_str}
    """

In [ ]:
%%time
# check distribution of days_supply, quantity, unit_cost and cost

days_supply = get_dist_cts("days_supply", rx_enc_step1_1p)
days_supply_pos = get_dist_cts("days_supply", rx_enc_step1_1p, filter = "days_supply > 0")
days_supply_clean = get_dist_cts("days_supply_clean", rx_enc_step2_1p)

quantity_dispensed = get_dist_cts("quantity_dispensed", rx_enc_step1_1p)
quantity_dispensed_pos = get_dist_cts("quantity_dispensed", rx_enc_step1_1p, filter = "quantity_dispensed > 0")
quantity_final = get_dist_cts("quantity_final", rx_enc_step2_1p)

plan_pay_actual = get_dist_cts("plan_pay_actual", rx_enc_impute_1p)
total_cost_actual = get_dist_cts("total_cost_actual", rx_enc_impute_1p)
plan_pay_clean = get_dist_cts("plan_pay_clean", rx_enc_impute_1p)
total_cost_clean = get_dist_cts("total_cost_clean", rx_enc_impute_1p)
cost_impute = get_dist_cts("cost_impute", rx_enc_impute_1p)
cost_impute_adj = get_dist_cts("cost_impute_adj", rx_enc_impute_1p)
cost_final = get_dist_cts("cost_final", rx_enc_impute_1p)
cost_ds_impute = get_dist_cts("cost_ds_impute", rx_enc_impute_1p)

median_unit_cost = get_dist_cts("median_per_unit", unit_price)
nadac_unit_cost = get_dist_cts("nadac_per_unit", unit_price)
asp_unit_cost = get_dist_cts("asp_per_unit", unit_price)
final_unit_cost = get_dist_cts("unit_price_final", unit_price)

sql = f"""
    {days_supply}
    union all
    {days_supply_pos}
    union all
    {days_supply_clean}
    union all
    {quantity_dispensed}
    union all
    {quantity_dispensed_pos}
    union all
    {quantity_final}
    union all
    {plan_pay_actual}
    union all
    {plan_pay_clean}
    union all
    {cost_impute}
    union all
    {cost_impute_adj}
    union all
    {cost_final}
    union all    
    {cost_ds_impute}
    union all
    {median_unit_cost}
    union all  
    {nadac_unit_cost}
    union all
    {asp_unit_cost}
    union all
    {final_unit_cost}
;
"""
df_dist = read_sql(sql)
display(df_dist)

In [ ]:
%%time
# final cost distribution by year
sql = f"""
    {get_dist_cts("cost_final", rx_enc_impute_1p, group_by_list = ['year(claim_date)'])}
;
"""
df_dist = read_sql(sql)
display(df_dist)

In [ ]:
%%time
# final cost distribution by year
sql = f"""
    {get_dist_cts("cost_impute", rx_enc_impute_1p, group_by_list = ['year(claim_date)'])}
;
"""
df_dist = read_sql(sql)
display(df_dist)

In [ ]:
# check frequency of imputation performed
sql = f"""
    select cost_final_source, count(*) as n_claim, count(distinct ndc) as n_ndc
    from {rx_enc_impute}_1p
    group by cost_final_source
    order by cost_final_source
    ;
"""

df = read_sql(sql)
display(df)

sql = f"""
    select unit_price_source, count(*) as n_claim, count(distinct ndc) as n_ndc
    from {rx_enc_impute}_1p
    group by unit_price_source
    order by unit_price_source
    ;
"""

df = read_sql(sql)
display(df)

sql = f"""
    select quantity_source, count(*) as n_claim, count(distinct ndc) as n_ndc
    from {rx_enc_impute}_1p
    group by quantity_source
    order by quantity_source
    ;
"""

df = read_sql(sql)
display(df)

sql = f"""
    select cost_final_source, unit_price_source, quantity_source, count(*) as n_claim, count(distinct ndc) as n_ndc
    from {rx_enc_impute}_1p
    group by cost_final_source, unit_price_source, quantity_source
    order by cost_final_source, unit_price_source, quantity_source
    ;
"""

df = read_sql(sql)
display(df)

In [ ]:
%%time
# final cost distribution by year
sql = f"""
    select cost_final_source, unit_price_source, quantity_source, count(*) as n_claim, count(distinct ndc) as n_ndc
    from {rx_enc_impute_1p}
    where cost_final = 0
    group by cost_final_source, unit_price_source, quantity_source
    order by cost_final_source, unit_price_source, quantity_source
;
"""
df_dist = read_sql(sql)
display(df_dist)

In [ ]:
%%time
# compare imputed vs cleaned actual
sql = f"""
    with cost_diff as (
    select cost_impute - plan_pay_clean as diff, quantity_source, unit_price_source
    from {rx_enc_impute}_1p
    where cost_impute > 0 and plan_pay_clean > 0
    )
    {get_dist_cts("diff", 'cost_diff', group_by_list = ['quantity_source', 'unit_price_source'])}
    ;
"""

df = read_sql(sql)
display(df)

sql = f"""
    with cost_diff as (
    select (cost_impute - plan_pay_clean)/plan_pay_clean as relative_diff, quantity_source, unit_price_source
    from {rx_enc_impute}_1p
    where cost_impute > 0 and plan_pay_clean > 0
    )
    {get_dist_cts("relative_diff", 'cost_diff', group_by_list = ['quantity_source', 'unit_price_source'])}
    ;
"""

df = read_sql(sql)
display(df)

In [ ]:
%%time
# compare imputed vs cleaned actual
sql = f"""
    with cost_diff as (
    select cost_ds_impute - plan_pay_clean as diff
    from {rx_enc_impute}_1p
    where cost_ds_impute > 0 and plan_pay_clean > 0
    )
    {get_dist_cts("diff", 'cost_diff')}
    ;
"""

df = read_sql(sql)
display(df)


In [ ]:
# check price year and inflation adjustment
sql = f"""
    select price_year, count(*) as n_claim, count(distinct ndc) as n_ndc
    from {rx_enc_impute}_1p
    group by price_year
    order by price_year
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select price_year, count(*)
    from {unit_price}
    group by price_year
    order by price_year
    ;
"""

df = read_sql(sql)
display(df)

In [ ]:
# find a list of NDC with crazy large price or quite different from original
sql = f"""
    select o.ndc, dm.ndc, dm.active, dm.cui_l1_name, dm.brand_generic, n, avg_quantity, avg_plan_pay, avg_impute_adj, avg_cost_final
    from (
        select ndc, count(*) as n, 
            avg(quantity_final) as avg_quantity, 
            avg(plan_pay_clean) as avg_plan_pay, 
            avg(cost_impute_adj) as avg_impute_adj, 
            avg(cost_final) as avg_cost_final
        from {rx_enc_impute}
        where cost_final >= 100000 or cost_final_sentinel >= 100000
        group by ndc
    ) o
    left join {dm_file} dm
    on o.ndc = dm.ndc
    having dm.ndc is not null
    order by avg_cost_final desc
    ;
"""

df = read_sql(sql)
display(df)

In [ ]:
# find a list of NDC with crazy large price or quite different from original
sql = f"""
    select *
    from {rx_enc_impute}_1p
    where (cost_final > 100000 or cost_final_sentinel > 100000)
    order by cost_final desc
    ;
"""

# RX_ENCOUNTERS_20220511_IMPUTED_1P

df_ndc9 = read_sql(sql)
display(df_ndc9)

In [243]:
%%time
# a working template to study one particular type of drug to resolve outliers
sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where encounter_key = 'da4c2b065b1961320fce0fe61d6e6c3c'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {unit_price}
    where ndc = '54868075300'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where ndc = '54868075300' and plan_pay_actual > 0
    ;
"""
df = read_sql(sql)
display(df)

quantity_dispensed_ndc = get_dist_cts("quantity_dispensed", rx_enc_step1, filter = "ndc = '54868075300'", filter_print = 'ndc')
sql = f"""
    {quantity_dispensed_ndc}
    ;
"""
df = read_sql(sql)
display(df)

quantity_dispensed_ndc = get_dist_cts("PLAN_PAY_CLEAN", rx_enc_step1, filter = "ndc = '54868075300'", filter_print = 'ndc')
sql = f"""
    {quantity_dispensed_ndc}
    ;
"""
df = read_sql(sql)
display(df)

,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PAYER_TYPE_CODE,PATIENT_STATE,SOURCES,PHARMACY_SUBMITTED_COST,COPAY_COINSURANCE,PLAN_PAY_PRIMARY,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_10,QUANTITY_50,QUANTITY_90,QUANTITY_CLEAN,PLAN_PAY_10,PLAN_PAY_50,PLAN_PAY_90,PLAN_PAY_CLEAN,TOTAL_COST_10,TOTAL_COST_50,TOTAL_COST_90,TOTAL_COST_CLEAN
0,548680753,9QkqTapCVZsGxERvSe3W+XGIx+vo3XL6VydC42U6CN4=,da4c2b065b1961320fce0fe61d6e6c3c,2014-06-03,54868075300,548680753,1,1200000.0,None,M,GA,"[\n ""rowley""\n]",None,None,None,None,None,None,1,2.0,1200000.0,1200000.0,1200000.0,13.4,47.0,387.0,None,33.4,67.0,768.6,None


,NDC9,NDC,N_CLAIM,START_DATE,END_DATE,UNIT_PRICE_FINAL,UNIT_PRICE_SOURCE,PRICE_YEAR,MEDIAN_PER_UNIT,NADAC_PER_UNIT,ASP_PER_UNIT,MEDIAN_PLAN_PAY_PER_UNIT,MEDIAN_PLAN_PAY_PER_UNIT_BY_YEAR
0,548680753,54868075300,9,1970-01-01,2012-12-31,21.5,median,2013,21.5,None,None,21.5,24.75
1,548680753,54868075300,9,2016-01-01,2017-12-31,21.5,median,2016,21.5,None,None,21.5,18.50
2,548680753,54868075300,9,2020-01-01,2030-12-31,21.5,median,2019,21.5,None,None,21.5,20.50
3,548680753,54868075300,9,2013-01-01,2013-12-31,21.5,median,2013,21.5,None,None,21.5,24.75
4,548680753,54868075300,9,2014-01-01,2015-12-31,21.5,median_by_year,2014,21.5,None,None,21.5,21.50
5,548680753,54868075300,9,2018-01-01,2019-12-31,21.5,median,2018,21.5,None,None,21.5,20.50


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PAYER_TYPE_CODE,PATIENT_STATE,SOURCES,PHARMACY_SUBMITTED_COST,COPAY_COINSURANCE,PLAN_PAY_PRIMARY,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_10,QUANTITY_50,QUANTITY_90,QUANTITY_CLEAN,PLAN_PAY_10,PLAN_PAY_50,PLAN_PAY_90,PLAN_PAY_CLEAN,TOTAL_COST_10,TOTAL_COST_50,TOTAL_COST_90,TOTAL_COST_CLEAN


,VAR,FILTER,N,N_NULL,N_NEG,N_ZERO,N_POS,MEAN,VARIANCE,MIN,P_01,P_05,P_10,P_25,MEDIAN,P_75,P_90,P_95,P_99,MAX
0,quantity_dispensed,ndc,97,0,0,0,97,884639.917526,3.680113e+11,1.0,1.0,1.0,2.0,4.0,1200000.0,1200000.0,1200000.0,1200000.0,2400000.0,2400000.0


,VAR,FILTER,N,N_NULL,N_NEG,N_ZERO,N_POS,MEAN,VARIANCE,MIN,P_01,P_05,P_10,P_25,MEDIAN,P_75,P_90,P_95,P_99,MAX
0,PLAN_PAY_CLEAN,ndc,97,96,0,0,1,47.0,None,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0


CPU times: user 326 ms, sys: 4.18 ms, total: 330 ms
Wall time: 1min 11s


In [ ]:
# MISC investigations below

In [39]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select ndc9, count(distinct ndc) as n_ndc, VARIANCE(nadac_per_unit) as var
    from 
    (
    select left(ndc, 9) as ndc9, ndc, avg(nadac_per_unit) as nadac_per_unit
    from {nadac_per_unit_price}
    group by ndc9, ndc
    )
    group by ndc9
    having n_ndc > 1
    order by var desc
    limit 10
    ;
"""
df_ndc9 = read_sql(sql)
display(df_ndc9)

,NDC9,N_NDC,VAR
0,000851341,3,4451.410105
1,782060114,3,4451.410105
2,502420860,2,2365.455625
3,669930084,2,1026.400183
4,517590202,2,96.311331
5,005740820,2,75.768265
6,525360625,2,75.768265
7,690970802,2,75.768265
8,005740827,2,75.768265
9,674570409,2,65.270340


CPU times: user 12.7 ms, sys: 0 ns, total: 12.7 ms
Wall time: 1.37 s


In [42]:
sql = f"""
    select *
    from {nadac_per_unit_price}
    where left(ndc, 9) = '000851341'
    ;
"""
read_sql(sql)

,NDC,NADAC_PER_UNIT,NDC_DESCRIPTION,OTC,PRICING_UNIT,CLASSIFICATION_FOR_RATE_SETTING,START_DATE,END_DATE
0,00085134101,309.71889,ASMANEX TWISTHALR 220 MCG #120,N,EA,B,1970-01-01,2021-10-19
1,00085134101,309.71889,ASMANEX TWISTHALR 220 MCG #120,N,EA,B,2021-10-20,2022-01-13
2,00085134101,324.89512,ASMANEX TWISTHALR 220 MCG #120,N,EA,B,2022-01-14,2030-12-31
3,00085134102,215.36696,ASMANEX TWISTHALER 220 MCG #60,N,EA,B,1970-01-01,2021-12-21
4,00085134102,215.36696,ASMANEX TWISTHALER 220 MCG #60,N,EA,B,2021-12-22,2022-01-13
5,00085134102,225.91994,ASMANEX TWISTHALER 220 MCG #60,N,EA,B,2022-01-14,2030-12-31
6,00085134107,183.475,ASMANEX TWISTHALER 220 MCG #30,N,EA,B,1970-01-01,2021-12-21
7,00085134107,183.475,ASMANEX TWISTHALER 220 MCG #30,N,EA,B,2021-12-22,2022-01-13
8,00085134107,192.46528,ASMANEX TWISTHALER 220 MCG #30,N,EA,B,2022-01-14,2030-12-31


In [43]:
sql = f"""
    select *
    from {rx_enc_step1_1p}
    where ndc9 = '000851341'
    ;
"""
read_sql(sql)

,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,COPAY_COINSURANCE,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,gld39m0+mvcoc2gL1gny5pqPL+BcTFuTEmC9FvnyjGk=,9442283d58950263b5d34359c4a93f5e,2016-02-16,00085134107,000851341,30,1.0,None,NaN,0.0,...,NaN,166.0,166.0,30.0,3.0,1.0,513.0,166.0,549.0,166.0
1,Rdwhf/bkjzfkEJscGLBDkJ0K0dSD55LZeUNWmweDvTw=,428df8533f907ff57b74c7e20491a696,2019-07-10,00085134101,000851341,30,1.0,None,298.0,0.0,...,107.0,191.0,298.0,30.0,3.0,1.0,868.0,191.0,910.0,298.0
2,qZAW5tJCfd4rFSreZh0KbzS16qRYb3IhnRvDS2TZEFI=,70503acb87cef3bd67d201ea82877d6b,2019-05-09,00085134107,000851341,30,1.0,EA,231.0,45.0,...,45.0,143.0,188.0,30.0,3.0,1.0,513.0,143.0,549.0,188.0
3,qKMnCgvfOnoFuzTvgNhKMmBMyhE3rQ3Oe7cQxAoLq/E=,7f1e040a695fbd1aaac5925c612d26b8,2019-12-16,00085134102,000851341,90,3.0,EA,812.0,120.0,...,120.0,519.0,639.0,90.0,3.0,3.0,590.0,519.0,634.0,NaN
4,YtXBe/+/5cDMGN0ChcQVGJFwDs7HUzdYC0LbcbrWZH0=,1d592a694f8625b304a85dce5035513e,2020-03-17,00085134107,000851341,30,1.0,EA,231.0,NaN,...,0.0,188.0,188.0,30.0,3.0,1.0,513.0,188.0,549.0,188.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29277,3Ppu7N3JK7FZIlwYWqLThRYnAYdjh94tS1P5OrGA1/Q=,b974cb44c4600cac0e7115002c22731e,2016-07-05,00085134102,000851341,60,2.0,EA,484.0,10.0,...,10.0,394.0,404.0,60.0,3.0,2.0,590.0,394.0,634.0,404.0
29278,F//ajRBX4va4qQTgVGOqbzVTdptHmNALBKCIykAunik=,5939558705313331681596327f33d7b3,2016-12-28,00085134101,000851341,30,1.0,EA,366.0,NaN,...,0.0,322.0,322.0,30.0,3.0,1.0,868.0,322.0,910.0,322.0
29279,AmpmAXsw+RdM5pV3OHLcugyH3fxHvd/qxnzDaPCQVc8=,71a9fc0a9dc6f62bbfd402eeffc5b0c1,2017-01-25,00085134102,000851341,30,1.0,EA,252.0,30.0,...,30.0,178.0,208.0,30.0,3.0,1.0,590.0,178.0,634.0,208.0
29280,CrOxQsT4UeojCJ86hyWeQwbkkj21/VLL2vurufpQgik=,513b503b7970565d12ae9cd0712d8cb0,2021-01-28,00085134107,000851341,30,1.0,EA,NaN,30.0,...,NaN,NaN,NaN,30.0,3.0,1.0,513.0,NaN,549.0,NaN


In [54]:
sql = f"""
    select *
    from {nadac_per_unit_price}
    where left(ndc, 9) = '000744339'
    ;
"""
read_sql(sql)

,NDC,NADAC_PER_UNIT,NDC_DESCRIPTION,OTC,PRICING_UNIT,CLASSIFICATION_FOR_RATE_SETTING,START_DATE,END_DATE
0,00074433902,2904.5386399999998,HUMIRA PEN 40 MG/0.8 ML,N,EA,B,1970-01-01,2021-10-19
1,00074433902,2904.5386399999998,HUMIRA PEN 40 MG/0.8 ML,N,EA,B,2021-10-20,2022-01-13
2,00074433902,3119.4745,HUMIRA PEN 40 MG/0.8 ML,N,EA,B,2022-01-14,2030-12-31
3,00074433906,2900.60425,HUMIRA PEN CROHN'S-UC-HS STARTER 40 MG/0.8 ML,N,EA,B,1970-01-01,2020-12-31
4,00074433906,2900.60425,HUMIRA PEN CROHN'S-UC-HS STARTER 40 MG/0.8 ML,N,EA,B,2021-01-01,2022-01-13
5,00074433906,3115.2489600000004,HUMIRA PEN CROHN'S-UC-HS STARTER 40 MG/0.8 ML,N,EA,B,2022-01-14,2030-12-31
6,00074433907,2900.65499,HUMIRA PEN PSORIASIS-UVEITIS-ADOL HS STARTER 4...,N,EA,B,1970-01-01,2020-12-31
7,00074433907,2900.65499,HUMIRA PEN PSORIASIS-UVEITIS-ADOL HS STARTER 4...,N,EA,B,2021-01-01,2022-01-13
8,00074433907,3115.30346,HUMIRA PEN PSORIASIS-UVEITIS-ADOL HS STARTER 4...,N,EA,B,2022-01-14,2030-12-31


In [86]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select ndc9, PRICING_UNIT, count(distinct ndc) as n_ndc, VARIANCE(nadac_per_unit) as var
    from 
    (
    select left(ndc, 9) as ndc9, ndc, PRICING_UNIT, avg(nadac_per_unit) as nadac_per_unit
    from {nadac_per_unit_price}
    group by ndc9, ndc, PRICING_UNIT
    )
    group by ndc9, PRICING_UNIT
    order by var desc
    ;
"""
df_ndc9 = read_sql(sql)
display(df_ndc9)

,NDC9,PRICING_UNIT,N_NDC,VAR
0,637390969,EA,1,NaN
1,005170440,ML,1,NaN
2,707101230,EA,1,NaN
3,502420828,EA,1,NaN
4,707101609,EA,1,NaN
...,...,...,...,...
16538,683820010,EA,2,0.0
16539,633040090,EA,2,0.0
16540,623320056,EA,2,0.0
16541,586570703,EA,2,0.0


CPU times: user 267 ms, sys: 0 ns, total: 267 ms
Wall time: 888 ms


In [81]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select left(ndc, 9) as ndc9, ndc, avg(nadac_per_unit) as nadac_per_unit
    from {nadac_per_unit_price}
    where ndc9 = '167140294'
    group by ndc9, ndc
    ;
"""
df_ndc9 = read_sql(sql)
display(df_ndc9)

,NDC9,NDC,NADAC_PER_UNIT
0,167140294,16714029401,0.082952
1,167140294,16714029402,0.067085
2,167140294,16714029403,0.062205


CPU times: user 11.7 ms, sys: 196 µs, total: 11.9 ms
Wall time: 155 ms


In [84]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select left(ndc, 9) as ndc9, *
    from {nadac_per_unit_price}
    where ndc9 = '762820628'
    ;
"""
df_ndc9 = read_sql(sql)
display(df_ndc9)

,NDC9,NDC,NADAC_PER_UNIT,NDC_DESCRIPTION,OTC,PRICING_UNIT,CLASSIFICATION_FOR_RATE_SETTING,START_DATE,END_DATE
0,762820628,76282062810,0.0246,CITALOPRAM HBR 10 MG TABLET,N,EA,G,1970-01-01,2021-12-21
1,762820628,76282062810,0.0246,CITALOPRAM HBR 10 MG TABLET,N,EA,G,2021-12-22,2022-01-18
2,762820628,76282062810,0.026,CITALOPRAM HBR 10 MG TABLET,N,EA,G,2022-01-19,2022-02-22
3,762820628,76282062810,0.026369999999999998,CITALOPRAM HBR 10 MG TABLET,N,EA,G,2022-02-23,2030-12-31
4,762820628,76282062890,0.026,CITALOPRAM HBR 10 MG TABLET,N,EA,G,1970-01-01,2022-02-15
5,762820628,76282062890,0.026,CITALOPRAM HBR 10 MG TABLET,N,EA,G,2022-02-16,2022-02-22
6,762820628,76282062890,0.026369999999999998,CITALOPRAM HBR 10 MG TABLET,N,EA,G,2022-02-23,2030-12-31


CPU times: user 16.1 ms, sys: 0 ns, total: 16.1 ms
Wall time: 174 ms


In [44]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select pricing_unit, count(*)
    from {nadac_per_unit_price}
    group by pricing_unit
    ;
"""
df_ndc9 = read_sql(sql)
display(df_ndc9)

,PRICING_UNIT,COUNT(*)
0,ML,9079
1,EA,79524
2,GM,5993


CPU times: user 236 ms, sys: 0 ns, total: 236 ms
Wall time: 15.7 s


In [103]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select left(ndc, 9) as ndc9, *
    from {unit_price}
    where ndc = '00003021501'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where ndc = '00003021501'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where encounter_key = '2128c458187afdf637749d3eb97f2a17'
    ;
"""
df = read_sql(sql)
display(df)

,NDC9,NDC,N_CLAIM,START_DATE,END_DATE,UNIT_PRICE_FINAL,MEDIAN_TOTAL_COST_PER_UNIT,MEDIAN_PLAN_PAY_PER_UNIT,NADAC_PER_UNIT,ASP_PER_UNIT
0,000030215,00003021501,35923,1970-01-01,2030-12-31,748.785714,748.785714,748.785714,None,None


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,000030215,8V0F9MrqR321YIHuXg6Tcf8/ROdP0ZLatnSkBGrXxGA=,30a9925022a3b6931fa4fac7710f7b64,2016-06-03,00003021501,000030215,28,28.0,None,NaN,...,NaN,21105.0,21105.0,28,282.0,28.0,22228.0,21105.0,22315.0,21105.0
1,000030215,zmlpJdflbvQeR2ZNzArGOYF7oGRukBNgr9eSsUu7UWw=,df6ed94f146488e5f482040433e79f3c,2015-09-21,00003021501,000030215,28,28.0,None,24144.0,...,25.0,21181.0,21206.0,28,282.0,28.0,22228.0,21181.0,22315.0,21206.0
2,000030215,pzmOiQxXkM9EUccHlmKs+3Y3qoNZuUNc4LfCFY6KCDw=,18b0baf56033adc4db9e128e27f8a945,2015-11-09,00003021501,000030215,28,28.0,EA,25200.0,...,0.0,22090.0,22090.0,28,282.0,28.0,22228.0,22090.0,22315.0,22090.0
3,000030215,1UH29CqVLZtCERUrA3JsGEMuE9pxBqIQo0iK4ccHinE=,20df6fbb54395b57d4809a5662aaa2fe,2015-12-15,00003021501,000030215,28,28.0,None,25210.0,...,0.0,22180.0,22180.0,28,282.0,28.0,22228.0,22180.0,22315.0,22180.0
4,000030215,tR/JHyQOjdYSN2zHEkto4cyuUyD0GRaytbF3CwEUgKQ=,d9c9360e7f7b1ec67a1a58a17fdfd814,2016-01-08,00003021501,000030215,14,14.0,None,NaN,...,NaN,10238.0,10238.0,14,282.0,14.0,22228.0,10238.0,22315.0,10238.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,000030215,zQjWQPk/5K+EFeyd/GMKu3NIam1siArnLowfY4OvDVQ=,34002a6113765fa98fda06897ec41fee,2015-10-23,00003021501,000030215,28,28.0,None,NaN,...,NaN,NaN,NaN,28,282.0,28.0,22228.0,NaN,22315.0,NaN
382,000030215,HOqNq/UNs+jE7/unGKpA/ZxAxXsvQzNMHFpT3+/6S3Q=,76b343249331e4bbbfeb2c8fa67d51f1,2016-04-27,00003021501,000030215,28,28.0,None,25210.0,...,0.0,20954.0,20954.0,28,282.0,28.0,22228.0,20954.0,22315.0,20954.0
383,000030215,dSo/VEaThA95jXy3VHv+z54ZpKbjD58FdEcWhXvn26E=,cac36747da11fb2011e5fe649def9f15,2016-05-17,00003021501,000030215,28,28.0,None,NaN,...,NaN,21576.0,21576.0,28,282.0,28.0,22228.0,21576.0,22315.0,21576.0
384,000030215,oCAGCO+5pAhbqndOIFobR8GZywMpChZWG8IW4yyaEj4=,f1aa78d8e029e4c577d8c23579873334,2016-06-08,00003021501,000030215,28,28.0,EA,25205.0,...,0.0,20954.0,20954.0,28,282.0,28.0,22228.0,20954.0,22315.0,20954.0


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,000030215,ZE1tW75yB2uQJTeDoGGVdEgk13jwPobZ66L7BQwuy1s=,2128c458187afdf637749d3eb97f2a17,2016-05-20,00003021501,000030215,28,282.0,None,None,...,None,1039.0,1039.0,28,282.0,282.0,22228.0,1039.0,22315.0,1039.0


CPU times: user 567 ms, sys: 5.85 ms, total: 573 ms
Wall time: 893 ms


In [105]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select left(ndc, 9) as ndc9, *
    from {unit_price}
    where ndc = '55513028310'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where ndc = '55513028310'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where encounter_key = 'fca62d854ab6d9f734edc6b36d3e94c0'
    ;
"""
df = read_sql(sql)
display(df)

,NDC9,NDC,N_CLAIM,START_DATE,END_DATE,UNIT_PRICE_FINAL,MEDIAN_TOTAL_COST_PER_UNIT,MEDIAN_PLAN_PAY_PER_UNIT,NADAC_PER_UNIT,ASP_PER_UNIT
0,555130283,55513028310,971961,1970-01-01,2030-12-31,147.5,158.5,147.5,None,8.222


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,555130283,FOP+R2QF8GEE37DDpXqAXI1z6TKy3qsDPoDQKa6q4zY=,1e8dca71851f28b65b1c4fec8eca9a6a,2014-11-23,55513028310,555130283,1,60.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,60.0,2661.0,NaN,2713.0,NaN
1,555130283,h+PmwABfIQe2AG46BiX1XiJYs51ONfND9n6oz43DJDY=,4a4e823ffa0eabd0819e99e9ce682ec6,2016-05-21,55513028310,555130283,1,30.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,30.0,2661.0,NaN,2713.0,NaN
2,555130283,aMZKAsgHUrAAyjq35gqWwLbp5x3BH1OX2Z7KZdsY0fI=,351d2ed86348d4c3a9f9f72e168a8a5c,2016-10-26,55513028310,555130283,1,32.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,32.0,2661.0,NaN,2713.0,NaN
3,555130283,Dt5x/FivHm9pizk+Zg69e1xilI8xVQYtlTeAyQEL4kc=,9b4c8956a3337c4a121cda95bc457147,2019-03-04,55513028310,555130283,1,1655.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,1655.0,2661.0,NaN,2713.0,NaN
4,555130283,B3zvx0HPp8Vgs0wx7VGodNpqEUX4FPwpBccOkzUmL18=,1ea4959a38dbbc1ec79b43d6583a3a48,2019-10-08,55513028310,555130283,1,0.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,NaN,2661.0,NaN,2713.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,555130283,6l319f7HvuFiUeNAzY1CSK2Tx1d+5TyvtE0cvN2iQzs=,f9ebaf1332538e4a474a49b7ad2ac06a,2015-12-31,55513028310,555130283,1,10.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,10.0,2661.0,NaN,2713.0,NaN
680,555130283,FSjCsL09SKYs953sLyM9WzKCInpH2fPx5twYNumAH4M=,89a54cf4c48d729bd18fdd15f792313f,2016-03-07,55513028310,555130283,1,900.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,900.0,2661.0,NaN,2713.0,NaN
681,555130283,Rnor1MReMpyYe5Tlqm9oA0XxL/An9rIvvBezNchN3Do=,98031d76f9a50009c9df8d4bdbee6bc8,2016-03-30,55513028310,555130283,1,20.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,20.0,2661.0,NaN,2713.0,NaN
682,555130283,PEY2t1Fpuij0S9UFACCOKe2wZdZo2rQWjRwxylpD7ps=,0d5f456bce7bc88996a706aae57ddaee,2016-08-26,55513028310,555130283,1,50.0,None,NaN,...,NaN,NaN,NaN,1,5000.0,50.0,2661.0,NaN,2713.0,NaN


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,555130283,5WjGG+H/qei9Z64YAZhyc6jpuoyGiuBkCiTeZnLYBqA=,fca62d854ab6d9f734edc6b36d3e94c0,2016-03-02,55513028310,555130283,1,520.0,None,None,...,None,None,None,1,5000.0,520.0,2661.0,None,2713.0,None


CPU times: user 574 ms, sys: 0 ns, total: 574 ms
Wall time: 1.78 s


In [111]:
%%time
# ndc vs ndc9
# confirm if nadac unit price is the same
sql = f"""
    select left(ndc, 9) as ndc9, *
    from {unit_price}
    where ndc = '00562780505'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where ndc = '00562780505'
    ;
"""
df = read_sql(sql)
display(df)

sql = f"""
    select left(ndc, 9) as ndc9, *
    from {rx_enc_step1_1p}
    where encounter_key = '294858bbbf9d9702d4c5b699fb6b35df'
    ;
"""
df = read_sql(sql)
display(df)

,NDC9,NDC,N_CLAIM,START_DATE,END_DATE,UNIT_PRICE_FINAL,MEDIAN_TOTAL_COST_PER_UNIT,MEDIAN_PLAN_PAY_PER_UNIT,NADAC_PER_UNIT,ASP_PER_UNIT
0,005627805,00562780505,75514,1970-01-01,2030-12-31,105.0,109.0,105.0,None,78.989


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,005627805,vq4HvyOmvaVUlFPJLhisvn9x6LkuvmjQBR2+EGhtWxk=,8b82623f2978c566e1430a7699cad64d,2018-04-19,00562780505,005627805,30,1.0,None,NaN,...,NaN,NaN,NaN,30,1500.0,1.0,133.0,NaN,206.15,NaN
1,005627805,UYYb2X6md3zLnJiuRKM33EUWD/8PyQa0EPBmEqI5RAs=,4a172c5d5dc4886e429925f10b7168f9,2021-07-14,00562780505,005627805,1,1.0,None,NaN,...,NaN,NaN,NaN,1,1500.0,1.0,133.0,NaN,206.15,NaN
2,005627805,GxxI89iVkIEriFXwoLJLOf5nigsJjsqjkEFwnZa157c=,fc10ca78ecbb4d8ba774ecc649264a48,2021-12-29,00562780505,005627805,1,1.0,EA,160.0,...,0.0,118.0,118.0,1,1500.0,1.0,133.0,118.0,206.15,118.0
3,005627805,X2GcdBqfpBApgIWmIRBX++J64BlXOY9bdIEIWu9zRZo=,b00ae05bff0547061c66239232eeacea,2017-07-06,00562780505,005627805,1,1500.0,None,NaN,...,NaN,NaN,NaN,1,1500.0,1500.0,133.0,NaN,206.15,NaN
4,005627805,2q1uYKBR6pzm0Z+gmU/qVKyk+v6nrZj7RRpGSIEuJI4=,207792ed5c64d461c5fb14d110d4a2f7,2018-03-12,00562780505,005627805,28,1.0,None,110.0,...,109.0,0.0,109.0,28,1500.0,1.0,133.0,NaN,206.15,109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,005627805,vAj3KlhHBQuJmkVGS/xa3ZBZrw/5FJYlOjuwGTekv4A=,c8a99f596aa795885a3f965416d363be,2016-09-08,00562780505,005627805,1,1.0,None,NaN,...,NaN,0.0,0.0,1,1500.0,1.0,133.0,NaN,206.15,NaN
214,005627805,4SsN1yOC4nnKoJUD7bz/DOtf4vLYyDNOTslBqrEsNfg=,946a49801960913ecb81c77b85595821,2018-03-16,00562780505,005627805,1,1500.0,None,NaN,...,NaN,NaN,NaN,1,1500.0,1500.0,133.0,NaN,206.15,NaN
215,005627805,Pb7Cji3mCN+fbpvpyJMxW8xdJSuhJNAq9LGmZlDmk20=,d679b28e3b22eddf51553c4428b3aea1,2021-07-13,00562780505,005627805,1,1500.0,None,NaN,...,NaN,NaN,NaN,1,1500.0,1500.0,133.0,NaN,206.15,NaN
216,005627805,AON0kXuXkr2Cc2EgWLDaFz6nd32u6s9Uxk9mmqw/N5I=,9d71efd4fbbb5d71b6739f4f93abd3ef,2021-10-18,00562780505,005627805,1,1.0,EA,160.0,...,0.0,118.0,118.0,1,1500.0,1.0,133.0,118.0,206.15,118.0


,NDC9,UPK_KEY2,ENCOUNTER_KEY,CLAIM_DATE,NDC,NDC9,DAYS_SUPPLY,QUANTITY_DISPENSED,UNIT_OF_MEASURE,PHARMACY_SUBMITTED_COST,...,PATIENT_PAY,PLAN_PAY_ACTUAL,TOTAL_COST_ACTUAL,DAYS_SUPPLY_CLEAN,QUANTITY_99,QUANTITY_CLEAN,PLAN_PAY_99,PLAN_PAY_CLEAN,TOTAL_COST_99,TOTAL_COST_CLEAN
0,003782046,OB4c7wXvbiBTM4PFXWuYFvGbfiD7w5rMMqw/g7wvFU0=,294858bbbf9d9702d4c5b699fb6b35df,2020-02-07,00378204601,003782046,30,180000.0,None,569.0,...,4.0,None,4.0,30,360000.0,180000.0,1049.25,None,951.0,4.0


CPU times: user 520 ms, sys: 0 ns, total: 520 ms
Wall time: 23 s
